In [13]:
import site
site.addsitedir("/var/www/data-prototyping/")

from datetime import date, datetime
from dateutil.relativedelta import relativedelta as rdt
import io

import bs4
import requests
import pandas as pd

from pygurujobs.job import job
from pygurujobs.jobbase import JobBase
from pygurucore.tables import GuruTable

import pdb

In [14]:
s=None

In [15]:
def scrape(month, t):
    count = nav_search(month)
    nav_down(month, count)
    for x in range((count - 1) // 500 + 1):
        down_insert(x, t, month)

In [34]:
def nav_search(month):
    mt = month.strftime('%b').upper()
    yr = month.strftime('%Y')
    b = s.post(
        'http://spring.ura.gov.sg//lad/ore/property_market/transaction/Transaction_Resi_Process.cfm',
        data={
            'Enbloc': 'YN',
            'contractMON_from': mt,
            'contractMON_to': mt,
            'contractYY_from': yr,
            'contractYY_to': yr,
            'locality': 'stzone',
            'search': 'projdetail',
            'sorting': 'CONTRACT_DATE',
            'submit_type': 'submit'
            }
        )
    soup = bs4.BeautifulSoup(b.content)
    return int(soup.find("input", {"name": "recordcount"}).attrs['value'])

In [17]:
def nav_down(month, count):
    end = (month + rdt(day=31))
    s.post(
        'http://spring.ura.gov.sg//lad/ore/property_market/transaction/Transaction_Resi_DisplayTable.cfm',
        data={
            'LineNbr': '1',
            'StartRow': '1',
            'caption': 'Contract Date|JAN 2015 - JAN 2015',
            'choice': 'download',
            'dispblk': 'AAAafbAA6AABxLBAAW,AAAafbAA6AABxLBAAX,AAAafbAA6AABxLBAAh,AAAafbAA6AABxLBAAg,AAAafbAA6AABxLBAAd',
            'downloaded_recs': ['AAAafbAA6AABxLBAAW', 'AAAafbAA6AABxLBAAX', 'AAAafbAA6AABxLBAAh', 'AAAafbAA6AABxLBAAg', 'AAAafbAA6AABxLBAAd'],
            'filter': " 1=1 and CONTRACT_DATE >= to_date('" + month.strftime('%d-%b-%Y').upper() + "','DD-MON-YYYY') and CONTRACT_DATE <= to_date('" + end.strftime('%d/%b/%Y').upper() + "','DD-MON-YYYY') ",
            'opt': 'project_name,house_no|street_name|storey|unit_no,cover,floor_land_area,lot_type,price,unit_price,unit_price_psf,actual_contract_date,property_type_code,title,completion_date,sale_type_ind,purchaser_address_ind,postal_code,postal_sector,postal_code1,region_code,dgp_code',
            'ordercriteria': 'contract_date desc',
            'recordcount': count,
            'selected_display': "project_name,house_no,initcap(street_name) street_name,storey,unit_no,cover,round(floor_land_area) floor_land_area,nvl(decode(lot_type,'L','Land','U','Strata'),'Unknown') as lot_type,round(price) price,round(price/floor_land_area) unit_price,round((price/floor_land_area)/10.764) unit_price_psf,actual_contract_date,property_type_code,decode(title,'NA','N.A.',initcap(title)) title,completion_date,decode(sale_type_ind,'1','New Sale','2','Sub Sale','3','Resale','O','Other','N.A.') sale_type_ind,decode(nvl(purchaser_address_ind, ' '), ' ', 'N.A.', purchaser_address_ind) purchaser_address_ind,postal_code,postal_sector,postal_code1,region_code,dgp_code",
            'sorting': 'CONTRACT_DATE',
            'text': 'Project Name,Address,No. of Units,Area (sqm),Type of Area,Transacted Price ($),Unit Price ($ psm),Unit Price ($ psf),Contract Date,Property Type,Tenure,Completion Date,Type of Sale,Purchaser Address Indicator,Postal District,Postal Sector,Postal Code,Planning Region,Planning Area',
            'title': "project_name,house_no,initcap(street_name) street_name,storey,unit_no,cover,round(floor_land_area) floor_land_area,nvl(decode(lot_type,'L','Land','U','Strata'),'Unknown') as lot_type,round(price) price,round(price/floor_land_area) unit_price,round((price/floor_land_area)/10.764) unit_price_psf,actual_contract_date,property_type_code,decode(title,'NA','N.A.',initcap(title)) title,completion_date,decode(sale_type_ind,'1','New Sale','2','Sub Sale','3','Resale','O','Other','N.A.') sale_type_ind,decode(nvl(purchaser_address_ind, ' '), ' ', 'N.A.', purchaser_address_ind) purchaser_address_ind,postal_code,postal_sector,postal_code1,region_code,dgp_code"
            }
        )

In [18]:
def down_insert(x, t, month):
    low = str(x * 500 + 1)
    up = str((x + 1) * 500)
    d = s.get(
        'http://spring.ura.gov.sg//lad/ore/property_market/transaction/transaction_download_CSV.cfm?filename=14012015160100%5F1%5Fallproperty%2Ecsv&lower=' + low + '&upper=' + up + '&downloadLimitRecs=10000&sorting=CONTRACT_DATE&tran_type=resi'
        )
    df = pd.read_csv(io.BytesIO(d.content), dtype=str).rename(columns={
            'Project Name': 'ura_key',
            'Address': 'address',
            'No. of Units': 'units',
            'Area (sqm)': 'area_sqm',
            'Type of Area': 'area_type',
            'Transacted Price ($)': 'transacted_price',
            'Unit Price ($ psm)': 'psm',
            'Unit Price ($ psf)': 'psf',
            'Contract Date': 'contract_date',
            'Property Type': 'property_type',
            'Tenure': 'tenure',
            'Completion Date': 'completion_year',
            'Type of Sale': 'sale_type',
            'Purchaser Address Indicator': 'purchaser',
            'Postal District': 'district',
            'Postal Sector': 'sector',
            'Postal Code': 'postcode',
            'Planning Region': 'planning_region',
            'Planning Area': 'planning_area'
        })
    pdb.set_trace()
    df['timestamp'] = month
    df.insert_sql(t, 1000, False)

In [29]:
def login():
    global s
    s = requests.Session()
    s.headers.update({
            'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
            'Accept-Encoding': 'gzip, deflate, sdch',
            'Accept-Language': 'en-US,en;q=0.8',
            'Connection': 'keep-alive',
            'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'
        })
    s.mount('http://', requests.adapters.HTTPAdapter(max_retries=5))
    s.post(
        'http://spring.ura.gov.sg/lad/ore/login/login_action.cfm',
        data={'Username': 'allproperty', 'password': 'lf456789', 'Submit': 'Login'},
        )
    try:
        s.cookies['CFID']
    except KeyError:
        raise Exception('Not able to login')

In [24]:
def logout():
    global s
    s.get('http://spring.ura.gov.sg/lad/ore/property_market/logout.cfm')

In [31]:
def exec_():
    login()
    start_month = "2015-07"
    months = 2
    try:
        t = GuruTable(table_name='stg_txn_ura_sales').truncate()
        try:
            last = datetime.strptime(start_month, '%Y-%m')
            last = last.date()
        except ValueError:
            last = date.today() - rdt(day=1)
        for x in range(months):
            try:
                scrape(last - rdt(months=x), t)
            except AttributeError:
                pass
            break
        print(last)
    finally:
        logout()

In [33]:
exec_()

<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 4.01 Transitional//EN" "http://www.w3.org/TR/html4/loose.dtd">
<html>
<head>
<script language="JavaScript" type="text/javascript">
<!-- Hide from old browsers
var imagenumber = 13 ;
var randomnumber = Math.random() ;
var rand1 = Math.round( (imagenumber-1) * randomnumber) + 1 ;
images = new Array
images[1] = "/lad/ore/login/images/scrnshots_realis_a.gif"
images[2] = "/lad/ore/login/images/scrnshots_realis_b.gif"
images[3] = "/lad/ore/login/images/scrnshots_realis_c.gif"
images[4] = "/lad/ore/login/images/scrnshots_realis_d.gif"
images[5] = "/lad/ore/login/images/scrnshots_realis_e.gif"
images[6] = "/lad/ore/login/images/scrnshots_realis_f.gif"
images[7] = "/lad/ore/login/images/scrnshots_realis_g.gif"
images[8] = "/lad/ore/login/images/scrnshots_realis_h.gif"
images[9] = "/lad/ore/login/images/scrnshots_realis_i.gif"
images[10] = "/lad/ore/login/images/scrnshots_realis_j.gif"
images[11] = "/lad/ore/login/images/scrnshots_realis_k.gif"
images[12] 